In [2]:
from sklearn.model_selection import KFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools

# ---- Load and Preprocess Data ----
train_dir = 'D:/Lung_cancer/train'  # Training directory path
target_size = (224, 224)  # Input size for VGG16
num_classes = 3

# Extract file paths and labels
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,  # Temporary batch size
    class_mode='categorical',
    shuffle=False  # To keep track of indices
)
X = np.array(train_generator.filepaths)  # File paths of images
y = np.array(train_generator.classes)    # Corresponding class labels

# ---- Hyperparameter Search Space ----
batch_sizes = [16, 32]
dropout_rates = [0.3, 0.5]
learning_rates = [0.0005, 0.001]
epochs_list = [10, 15, 20]  # Number of epochs
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_params = {}
best_accuracy = 0

# Iterate over all combinations of hyperparameters
for batch_size, dropout_rate, learning_rate, num_epochs in itertools.product(batch_sizes, dropout_rates, learning_rates, epochs_list):
    print(f"Testing batch_size={batch_size}, dropout={dropout_rate}, lr={learning_rate}, epochs={num_epochs}")
    
    fold_accuracies = []
    fold_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}")
        
        # Split into training and validation sets
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Load data dynamically
        def load_data(file_paths, labels):
            images = []
            for file in file_paths:
                img = load_img(file, target_size=target_size)
                img = img_to_array(img)
                images.append(img)
            images = np.array(images) / 255.0
            labels = np.eye(num_classes)[labels]
            return images, labels
        
        X_train_images, y_train_labels = load_data(X_train, y_train)
        X_val_images, y_val_labels = load_data(X_val, y_val)
        
        # ---- Define and Compile Model ----
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False
        
        x = base_model.output
        x = layers.Flatten()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        output = layers.Dense(num_classes, activation='softmax')(x)
        model = models.Model(inputs=base_model.input, outputs=output)
        
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # ---- Train Model ----
        history = model.fit(
            X_train_images, y_train_labels,
            validation_data=(X_val_images, y_val_labels),
            epochs=num_epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        # ---- Evaluate Model ----
        val_loss, val_accuracy = model.evaluate(X_val_images, y_val_labels, verbose=0)
        print(f"Fold {fold + 1} Accuracy: {val_accuracy * 100:.2f}%")
        print(f"Fold {fold + 1} Loss: {val_loss:.4f}")
        
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    # Compute mean accuracy for the current hyperparameter combination
    mean_accuracy = np.mean(fold_accuracies)
    mean_loss = np.mean(fold_losses)
    
    print(f"Mean Accuracy: {mean_accuracy * 100:.2f}%, Mean Loss: {mean_loss:.4f}\n")
    
    # Store best hyperparameters
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {
            "batch_size": batch_size,
            "dropout": dropout_rate,
            "learning_rate": learning_rate,
            "epochs": num_epochs,
            "accuracy": best_accuracy
        }

# ---- Final Best Parameters ----
print("Best Parameters:")
print(best_params)


Found 406 images belonging to 3 classes.
Testing batch_size=16, dropout=0.3, lr=0.0005, epochs=10

Fold 1
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.2737 - loss: 3.3714 - val_accuracy: 0.5366 - val_loss: 0.9921
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5563 - loss: 1.0610 - val_accuracy: 0.6341 - val_loss: 0.8180
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.6855 - loss: 0.6933 - val_accuracy: 0.7683 - val_loss: 0.6219
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.7884 - loss: 0.5234 - val_accuracy: 0.5610 - val_loss: 0.9513
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.7644 - loss: 0.5561 - val_accuracy: 0.7561 - val_loss: 0.5891
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.8502 - loss: 0.3931 - val_accuracy: 0.7439 - val_loss: 0.5480
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8930 - loss: 0.3321 - val_accuracy: 0.7561 - val_loss: 0.5288
Epoch 8/10
2

MemoryError: Unable to allocate 187. MiB for an array with shape (325, 224, 224, 3) and data type float32

In [2]:
from sklearn.model_selection import KFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools

# ---- Load and Preprocess Data ----
train_dir = 'D:/Lung_cancer/train'  # Training directory path
target_size = (224, 224)  # Input size for Xception
num_classes = 3

# Extract file paths and labels
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,  # Temporary batch size
    class_mode='categorical',
    shuffle=False  # To keep track of indices
)
X = np.array(train_generator.filepaths)  # File paths of images
y = np.array(train_generator.classes)    # Corresponding class labels

# ---- Hyperparameter Search Space ----
batch_sizes = [32]
dropout_rates = [0.3, 0.5]
learning_rates = [0.0005, 0.001]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_params = {}
best_accuracy = 0

# Iterate over all combinations of hyperparameters
for batch_size, dropout_rate, learning_rate in itertools.product(batch_sizes, dropout_rates, learning_rates):
    print(f"Testing batch_size={batch_size}, dropout={dropout_rate}, lr={learning_rate}")
    
    fold_accuracies = []
    fold_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}")
        
        # Split into training and validation sets
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Load data dynamically
        def load_data(file_paths, labels):
            images = []
            for file in file_paths:
                img = load_img(file, target_size=target_size)
                img = img_to_array(img)
                images.append(img)
            images = np.array(images) / 255.0
            labels = np.eye(num_classes)[labels]
            return images, labels
        
        X_train_images, y_train_labels = load_data(X_train, y_train)
        X_val_images, y_val_labels = load_data(X_val, y_val)
        
        # ---- Define and Compile Model ----
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False
        
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        output = layers.Dense(num_classes, activation='softmax')(x)
        model = models.Model(inputs=base_model.input, outputs=output)
        
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # ---- Train Model ----
        history = model.fit(
            X_train_images, y_train_labels,
            validation_data=(X_val_images, y_val_labels),
            epochs=10,
            batch_size=batch_size,
            verbose=1
        )
        
        # ---- Evaluate Model ----
        val_loss, val_accuracy = model.evaluate(X_val_images, y_val_labels, verbose=0)
        print(f"Fold {fold + 1} Accuracy: {val_accuracy * 100:.2f}%")
        print(f"Fold {fold + 1} Loss: {val_loss:.4f}")
        
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    # Compute mean accuracy for the current hyperparameter combination
    mean_accuracy = np.mean(fold_accuracies)
    mean_loss = np.mean(fold_losses)
    
    print(f"Mean Accuracy: {mean_accuracy * 100:.2f}%, Mean Loss: {mean_loss:.4f}\n")
    
    # Store best hyperparameters
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {
            "batch_size": batch_size,
            "dropout": dropout_rate,
            "learning_rate": learning_rate,
            "accuracy": best_accuracy
        }

# ---- Final Best Parameters ----
print("Best Parameters:")
print(best_params)


Found 406 images belonging to 3 classes.
Testing batch_size=32, dropout=0.3, lr=0.0005

Fold 1
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.3560 - loss: 1.1743 - val_accuracy: 0.5000 - val_loss: 0.9729
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.4449 - loss: 1.0505 - val_accuracy: 0.6463 - val_loss: 0.9516
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5056 - loss: 0.9757 - val_accuracy: 0.6098 - val_loss: 0.8906
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5617 - loss: 0.9378 - val_accuracy: 0.6341 - val_loss: 0.8524
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.5639 - loss: 0.9256 - val_accuracy: 0.6707 - val_loss: 0.8419
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.5858 - loss: 0.9127 - val_accuracy: 0.6585 - val_loss: 0.8066
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.6222 - loss: 0.8772 - val_accuracy: 0.6951 - val_loss: 0.7941
Epoch 8/10
11/11 ━━━━━━

In [ ]:
Testing batch_size=16, dropout=0.5, lr=0.0005